# Bounding Cases for OreSat Temperature Field

## Intro

### References

- [Table of some emissivities][epsilon]
- [NASA paper on PCBs in space][NASA PCB]

[epsilon]: http://www.solarmirror.com/fom/fom-serve/cache/43.html
[NASA PCB]: https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19920002003.pdf

### Imports

In [130]:
library('units')
getName <- function(name, object) 
    # Bring the unit name from ud_units into the current environment
    if (  !is.null( object[[name]] )  ) 
    {
        assign(
            x=name, 
            value=object[[name]], 
            envir=parent.env(environment())
            )
    } else  warning('Name ', name, ' not found in ud_units.')
        
sapply( 
    c('m','km','mm','W','K','rad','sr'), 
    function(n) getName(n, ud_units) 
    )
ul <- m/m # easy way to specify unitless quantities
deg <- make_unit(name='deg')
install_conversion_constant(from='deg', to='rad', const=2*pi/360)

m  km  mm   W   K rad  sr 
  1   1   1   1   1   1   1

### input parameters

### physical values

In [101]:
area_circ <- function(r) pi*r^2
area_sphere <- function(r) 4*pi*r^2
volume_sphere <- function(r) 4/3*pi*r^3
solidAngle <- function(theta) 2*pi*(1*ul-cos(theta))  
sigma= 5.67e-8 *W/m^2/K^4 # Stefan-Boltzman constant
dist_sol= 1.496e8 *km
rad_sol= 6.96392e5 *km
# theta_halfSun= atan((rad_sol/dist_sol)) # for educational purposes
# cat(solidAngle(theta_halfSun))
view_sol= area_circ(rad_sol)/area_sphere(dist_sol)*2 *ul # sun's view factor (nice approximation)
rad_earth= 6.371e3 *km
altitude= 404 *km
# Ignore the warning about asin(). This is exactly how that function is supposed to be used.
theta_halfEarth= asin( rad_earth/(rad_earth+altitude) ) *rad
view_earth= solidAngle(theta_halfEarth)/2/pi
rho_sol_earth= 0.367 *ul # reflectance of earth based on sunlight (albedo)
T_ice= 273.2 *K
T_room= T_ice+ 20*K
T_sol= 5.773e3 *K
T_earth= 14*K +T_ice
T_space= 2.7 *K
# emissivities, see [epsilon]
eps_bb_al= 0.84 *ul # average black body emissivity of anodized aluminum
eps_sol_al= 0.14 *ul # average solar emissivity of anodized aluminum
eps_al_black= 0.88 *ul # average (solar and black body) emissivity of black anodized aluminum

Warning message in Math.units(rad_earth/(rad_earth + altitude)):
"Operation asin not meaningful for units"

In [116]:
radTransfer <- function(viewFactor, emissivity, T)
    emissivity*viewFactor*sigma*T^4
radTransfer(view_sol, 1, T_sol)
radTransfer(1, 1, T_earth)
q.sol <- 4*pi*rad_sol^2*sigma*T_sol^4
qA.sol <- q.sol/(4*pi*dist_sol^2)
units(qA.sol) <- W/m^2
qA.sol
qA.earth <- qA.sol*rho_sol_earth
qA.earth

682.3444 W/m^2

390.0794 W/m^2

1364.689 W/m^2

500.8408 W/m^2

## Worst Case

+Y or -Y face is pointed directly at the sun. +/-Y sides act as heat sinks. The +/-Z beams of the front/back frames are thin fins connecting the sides. OreSat is directly between Earth and Sol. The sides are considered to be planes (neglecting the widths of the rails and frames).

In [120]:
# 0= qA.sol*eps_sol_al -eps_bb_al*sigma*T_sunside^4
# T_sunside^4= qA.sol*eps_sol_al/(eps_bb_al*sigma)
T_sunside= ( qA.sol*eps_sol_al/(eps_bb_al*sigma) )^(1/4)
T_sunside
T_earth=  ( qA.sol*(1*ul-rho_sol_earth)/(0.9*sigma) )^(1/4)
units(T_earth) <- 

251.6662 K

360.7057 K

In [107]:
equil_nearsun_steadyF <- function(T) radTransfer(view_sol, 1*ul, T_sol) - radTransfer(1*ul,1*ul, T)
asdf <- uniroot(equil_nearsun_steadyF, lower= T_space, upper= T_sol)
# T_nearsun_steady <- 

# T_farsun_init= ???

ERROR: Error in Ops.units(mmm, each): both operands of the expression should be "units" objects


## Best Case